In [3]:
import javalang
import pandas as pd
from tqdm import tqdm
import numpy as np

In [4]:
df = pd.read_csv('/content/NonFullContext-Result.csv',index_col='index')

In [5]:
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
                
    return (matrix[size_x - 1, size_y - 1])


def levenshtein_normalized(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return matrix[size_x - 1, size_y - 1]/max(size_x, size_y)

In [6]:
target = list(df['body'])
original = list(df['generatedResultOriginal'])
perturbated0 = list(df['generatedResultP0'])
perturbated1 = list(df['generatedResultP1'])
perturbated2 = list(df['generatedResultP2'])

resultOriginalMethods = list(df['mvnTestResultOriginal'])
resultP0Methods = list(df['mvnTestResultP0'])
resultP1Methods = list(df['mvnTestResultP1'])
resultP2Methods = list(df['mvnTestResultP2'])


###############################################

originalJavaDoc = list(df['javaDocFirstSentence'])
perturbatedJavaDoc0 = []
perturbatedJavaDoc1 = []
perturbatedJavaDoc2 = []
for item in df['perturbatedJavaDocSentence']:
  item = eval(item)
  perturbatedJavaDoc0.append(item[0])
  perturbatedJavaDoc1.append(item[1])
  perturbatedJavaDoc2.append(item[2])

In [ ]:
#Lev distance computation for code

levDistanceTargetOriginal = []
levDistanceTargetP0 = []
levDistanceTargetP1 = []
levDistanceTargetP2 = []
levDistanceTargetOriginalNorm = []
levDistanceTargetP0Norm = []
levDistanceTargetP1Norm = []
levDistanceTargetP2Norm = []

levDistanceOriginalP0 = []
levDistanceOriginalP1 = []
levDistanceOriginalP2 = []
levDistanceOriginalP0Norm = []
levDistanceOriginalP1Norm = []
levDistanceOriginalP2Norm = []

for (targetMethod, originalMethod, perturbated0Method, perturbated1Method, perturbated2Method, resultOriginalMethod, resultP0Method, resultP1Method, resultP2Method) in tqdm(zip(target, original, perturbated0, perturbated1, perturbated2, resultOriginalMethods, resultP0Methods, resultP1Methods, resultP2Methods)):

  originalBodyTokens = []
  targetBodyTokens = []

  if originalMethod == 'Empty Method' or resultOriginalMethod == 'Not Valid' or resultOriginalMethod == 'Syntax Error':
      
      # levDistanceOriginalP0.append('None')
      # levDistanceOriginalP0Norm.append('None')

      # levDistanceOriginalP1.append('None')
      # levDistanceOriginalP1Norm.append('None')
      
      # levDistanceOriginalP2.append('None')
      # levDistanceOriginalP2Norm.append('None')

      levDistanceTargetOriginal.append('None')
      levDistanceTargetOriginalNorm.append('None')
     
  
  else:
    originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
    targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

    lev = levenshtein(originalBodyTokens, targetBodyTokens)
    levDistanceTargetOriginal.append(lev)

    levNorm = levenshtein_normalized(originalBodyTokens, targetBodyTokens)
    levDistanceTargetOriginalNorm.append(levNorm)

  ##################################################################################################
  
  if perturbated0Method == 'Empty Method' or resultP0Method == 'Not Valid' or resultP0Method == 'Syntax Error':
    
    levDistanceOriginalP0.append('None')
    levDistanceOriginalP0Norm.append('None')

    levDistanceTargetP0.append('None')
    levDistanceTargetP0Norm.append('None')
  
  else:
    try:
      originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
      p0BodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbated0Method))]
      targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

      lev = levenshtein(originalBodyTokens, p0BodyTokens)
      levDistanceOriginalP0.append(lev)

      levNorm = levenshtein_normalized(originalBodyTokens, p0BodyTokens)
      levDistanceOriginalP0Norm.append(levNorm)

      lev = levenshtein(targetBodyTokens, p0BodyTokens)
      levDistanceTargetP0.append(lev)

      levNorm = levenshtein_normalized(targetBodyTokens, p0BodyTokens)
      levDistanceTargetP0Norm.append(levNorm)

    except Exception:
      
      levDistanceOriginalP0.append('None')
      levDistanceOriginalP0Norm.append('None')

      levDistanceTargetP0.append('None')
      levDistanceTargetP0Norm.append('None')

  ##################################################################################################

  if perturbated1Method == 'Empty Method' or resultP1Method == 'Not Valid' or perturbated1Method == 'Syntax Error':
      
      levDistanceOriginalP1.append('None')
      levDistanceOriginalP1Norm.append('None')

      levDistanceTargetP1.append('None')
      levDistanceTargetP1Norm.append('None')

  else:          
      try:
        originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
        p1BodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbated1Method))]
        targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

        lev = levenshtein(originalBodyTokens, p1BodyTokens)
        levDistanceOriginalP1.append(lev)

        levNorm = levenshtein_normalized(originalBodyTokens, p1BodyTokens)
        levDistanceOriginalP1Norm.append(levNorm)

        lev = levenshtein(targetBodyTokens, p1BodyTokens)
        levDistanceTargetP1.append(lev)

        levNorm = levenshtein_normalized(targetBodyTokens, p1BodyTokens)
        levDistanceTargetP1Norm.append(levNorm)


      except Exception:

        levDistanceOriginalP1.append('None')
        levDistanceOriginalP1Norm.append('None')

        levDistanceTargetP1.append('None')
        levDistanceTargetP1Norm.append('None')

  ##################################################################################################
  if perturbated2Method == 'Empty Method' or resultP2Method == 'Not Valid' or perturbated2Method == 'Syntax Error':
      
      levDistanceOriginalP2.append('None')
      levDistanceOriginalP2Norm.append('None')

      levDistanceTargetP2.append('None')
      levDistanceTargetP2Norm.append('None')
  
  else:
      try:
        originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
        p2BodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbated2Method))]
        targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

        lev = levenshtein(originalBodyTokens, p2BodyTokens)
        levDistanceOriginalP2.append(lev)

        levNorm = levenshtein_normalized(originalBodyTokens, p2BodyTokens)
        levDistanceOriginalP2Norm.append(levNorm)

        lev = levenshtein(targetBodyTokens, p2BodyTokens)
        levDistanceTargetP2.append(lev)

        levNorm = levenshtein_normalized(targetBodyTokens, p2BodyTokens)
        levDistanceTargetP2Norm.append(levNorm)



      except Exception:
        
        levDistanceOriginalP2.append('None')
        levDistanceOriginalP2Norm.append('None')
        
        levDistanceTargetP2.append('None')
        levDistanceTargetP2Norm.append('None')
  ##################################################################################################


len(levDistanceOriginalP0)




In [8]:
 #Lev distance computation for javadoc

levDistanceOriginalJavaDocP0 = []
levDistanceOriginalJavaDocP1 = []
levDistanceOriginalJavaDocP2 = []

levDistanceOriginalJavaDocP0Norm = []
levDistanceOriginalJavaDocP1Norm = []
levDistanceOriginalJavaDocP2Norm = []

for (javaDocOriginal, javaDocP0, javaDocP1, javaDocP2) in tqdm(zip(originalJavaDoc, perturbatedJavaDoc0, perturbatedJavaDoc1, perturbatedJavaDoc2)):

    levDistanceOriginalJavaDocP0.append(levenshtein(javaDocOriginal.split(), javaDocP0.split()))
    levDistanceOriginalJavaDocP0Norm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocP0.split()))

    levDistanceOriginalJavaDocP1.append(levenshtein(javaDocOriginal.split(), javaDocP1.split()))
    levDistanceOriginalJavaDocP1Norm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocP1.split()))
    
    levDistanceOriginalJavaDocP2.append(levenshtein(javaDocOriginal.split(), javaDocP2.split()))
    levDistanceOriginalJavaDocP2Norm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocP2.split()))

1006it [00:02, 498.87it/s]


In [ ]:
codeBleuAgainstOriginal = []
codeBleuAgainstP0 = []
codeBleuAgainstP1 = []
codeBleuAgainstP2 = []


for (targetMethod, originalMethod, perturbated0Method, perturbated1Method, perturbated2Method, resultOriginalMethod, resultP0Method, resultP1Method, resultP2Method) in tqdm(zip(target, original, perturbated0, perturbated1, perturbated2, resultOriginalMethods, resultP0Methods, resultP1Methods, resultP2Methods)):

  if originalMethod == 'Empty Method' or resultOriginalMethod == 'Not Valid' or resultOriginalMethod == 'Syntax Error':
      codeBleuAgainstOriginal.append('None')
  else:
      
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          originalBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(originalMethod))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          
          with open('prediction.txt','w') as f:
            f.write(originalBodyTokens)
          
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
        
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
        
          codeBleuAgainstOriginal.append('{}'.format(codeBleuScore))
      
      except Exception:
        codeBleuAgainstOriginal.append('None')
        

  if perturbated0Method == 'Empty Method' or resultP0Method == 'Not Valid' or resultP0Method == 'Syntax Error':
      codeBleuAgainstP0.append('None')
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          p0BodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbated0Method))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(p0BodyTokens)
          
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
      
          infoScore = res[0]

          codeBleuScore = res[1].split(':')[1].split(',')[0]

          codeBleuAgainstP0.append('{}'.format(codeBleuScore))

      except Exception:
        codeBleuAgainstP0.append('None')
        

  if perturbated1Method == 'Empty Method' or resultP1Method == 'Not Valid' or resultP1Method == 'Syntax Error':
      codeBleuAgainstP1.append('None')
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          p1BodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbated1Method))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(p1BodyTokens)
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
      
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
      
          codeBleuAgainstP1.append('{}'.format(codeBleuScore))


      except Exception:
          codeBleuAgainstP1.append('None')
      
  if perturbated2Method == 'Empty Method' or resultP2Method == 'Not Valid' or resultP2Method == 'Syntax Error':
      codeBleuAgainstP2.append('None')
  
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          p2BodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbated2Method))])
      
          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(p2BodyTokens)
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
          
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
          codeBleuAgainstP2.append('{}'.format(codeBleuScore))
      
      except Exception:
          codeBleuAgainstP2.append('None')
      
      
      
      
     

In [10]:
df['CodeBleuOriginal'] = codeBleuAgainstOriginal
df['CodeBleuP0'] = codeBleuAgainstP0
df['CodeBleuP1'] = codeBleuAgainstP1
df['CodeBleuP2'] = codeBleuAgainstP2

df['levenshteinTarget-Original-Code'] =  levDistanceTargetOriginal
df['levenshteinTarget-Original-Code-Normalized'] =  levDistanceTargetOriginalNorm

df['levenshteinTarget-P0-Code'] =  levDistanceTargetP0
df['levenshteinTarget-P0-Code-Normalized'] =  levDistanceTargetP0Norm

df['levenshteinTarget-P1-Code'] =  levDistanceTargetP1
df['levenshteinTarget-P1-Code-Normalized'] =  levDistanceTargetP1Norm

df['levenshteinTarget-P2-Code'] =  levDistanceTargetP2
df['levenshteinTarget-P2-Code-Normalized'] =  levDistanceTargetP2Norm

######################################################################


df['levenshteinOriginal-P0-Code'] = levDistanceOriginalP0
df['levenshteinOriginal-P0-Code-Normalized'] = levDistanceOriginalP0Norm

df['levenshteinOriginal-P1-Code'] = levDistanceOriginalP1
df['levenshteinOriginal-P1-Code-Normalized'] = levDistanceOriginalP1Norm


df['levenshteinOriginal-P2-Code'] = levDistanceOriginalP2
df['levenshteinOriginal-P2-Code-Normalized'] = levDistanceOriginalP2Norm

######################################################################

df['levenshteinOriginal-P0-JavaDoc'] = levDistanceOriginalJavaDocP0
df['levenshteinOriginal-P0-JavaDoc-Normalized'] = levDistanceOriginalJavaDocP0Norm

df['levenshteinOriginal-P1-JavaDoc'] = levDistanceOriginalJavaDocP1
df['levenshteinOriginal-P1-JavaDoc-Normalized'] = levDistanceOriginalJavaDocP1Norm

df['levenshteinOriginal-P2-JavaDoc'] = levDistanceOriginalJavaDocP2
df['levenshteinOriginal-P2-JavaDoc-Normalized'] = levDistanceOriginalJavaDocP2Norm



In [11]:
# df.to_csv('Non-FullContext-Result1.csv')